# NER

In [ ]:
import pandas as pd
data = pd.read_csv('/content/ner_dataset.csv', encoding= 'unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}

    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))

    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [ ]:
# data['Word_idx'] = data['Word'].map(token2idx)
# data['Tag_idx'] = data['Tag'].map(tag2idx)
# data_fillna = data.fillna(method='ffill', axis=0)
# # Groupby and collect columns
# data_group = data_fillna.groupby(
# ['Sentence #'],as_index=False
# )['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [ ]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data_fillna = data.fillna(method='ffill', axis=0)
# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)[['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx']].agg(lambda x: list(x))

<ipython-input-4-81c9403261f2>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_fillna = data.fillna(method='ffill', axis=0)


In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )

    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)


train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [ ]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [ ]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))

    #Optimiser
    # adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [ ]:
# results = pd.DataFrame()
# model_bilstm_lstm = get_bilstm_lstm_model()
# plot_model(model_bilstm_lstm)
# results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()

model_bilstm_lstm.build(input_shape=(None, input_length))

plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

26/26 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.8268 - loss: 1.8116 - val_accuracy: 0.9681 - val_loss: 0.3158
26/26 ━━━━━━━━━━━━━━━━━━━━ 25s 977ms/step - accuracy: 0.9676 - loss: 0.3158 - val_accuracy: 0.9681 - val_loss: 0.2240
26/26 ━━━━━━━━━━━━━━━━━━━━ 25s 970ms/step - accuracy: 0.9676 - loss: 0.2689 - val_accuracy: 0.9681 - val_loss: 0.2178
26/26 ━━━━━━━━━━━━━━━━━━━━ 25s 964ms/step - accuracy: 0.9676 - loss: 0.2573 - val_accuracy: 0.9681 - val_loss: 0.2137
26/26 ━━━━━━━━━━━━━━━━━━━━ 25s 971ms/step - accuracy: 0.9676 - loss: 0.2571 - val_accuracy: 0.9681 - val_loss: 0.1999
26/26 ━━━━━━━━━━━━━━━━━━━━ 25s 980ms/step - accuracy: 0.9676 - loss: 0.2271 - val_accuracy: 0.9681 - val_loss: 0.1752
26/26 ━━━━━━━━━━━━━━━━━━━━ 25s 966ms/step - accuracy: 0.9676 - loss: 0.2073 - val_accuracy: 0.9681 - val_loss: 0.1586
26/26 ━━━━━━━━━━━━━━━━━━━━ 26s 991ms/step - accuracy: 0.9677 - loss: 0.1918 - val_accuracy: 0.9681 - val_loss: 0.1503
26/26 ━━━━━━━━━━━━━━━━━━━━ 25s 952ms/step - accuracy: 0.967

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')


In [ ]:
text = nlp('''Today, in an operation in Narayanpur, Chhattisgarh, our security forces have neutralized 27 dreaded Maoists, including Nambala Keshav Rao, alias Basavaraju, the general secretary of CPI (Maoist), topmost leader, and the backbone of the Naxal movement,” Shah said in a post on X.

“This is the first time in three decades of Bharat’s battle against Naxalism that a general secretary-ranked leader has been neutralized by our forces. I applaud our brave security forces and agencies for this major breakthrough,” he said.

Prime Minister Narendra Modi also complimented the security forces for the “remarkable success”.

Basavaraju, who had trained to be an engineer before had enrolled for a masters when he dropped out, was considered a master Maoist strategist and had a cumulative reward of ₹1.5 crore on his head. According to a 2011 intelligence report, he was known to have always carried an AK-47.

''')
displacy.render(text, style = 'ent', jupyter=True)

# Refined

In [ ]:
# ------------------------------
# Step 1: Load Libraries and Dataset
# ------------------------------
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.model_selection import train_test_split

# Load dataset
data = pd.read_csv('/content/ner_dataset.csv', encoding='unicode_escape')
data['Sentence #'] = data['Sentence #'].fillna(method='ffill')

# ------------------------------
# Step 2: Prepare Token and Tag Mappings
# ------------------------------
def get_dict_map(data, token_or_tag):
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    idx2tok = {idx: tok for idx, tok in enumerate(vocab)}
    tok2idx = {tok: idx for idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok

token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

# Map tokens and tags
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)

# Group by sentences
data_group = data.groupby(['Sentence #'], as_index=False)[['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx']].agg(lambda x: list(x))

# ------------------------------
# Step 3: Padding and Data Splitting
# ------------------------------
def get_pad_train_test_val(data_group, data):
    n_token = len(token2idx)
    n_tag = len(tag2idx)

    tokens = data_group['Word_idx'].tolist()
    tags = data_group['Tag_idx'].tolist()
    maxlen = max(len(s) for s in tokens)

    pad_tokens = pad_sequences(tokens, maxlen=maxlen, padding='post', value=n_token - 1)
    pad_tags = pad_sequences(tags, maxlen=maxlen, padding='post', value=tag2idx['O'])
    pad_tags = [to_categorical(i, num_classes=n_tag) for i in pad_tags]

    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_, tags_, test_size=0.25, random_state=2020)

    print(f"Train: {len(train_tokens)}, Val: {len(val_tokens)}, Test: {len(test_tokens)}")
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags, maxlen

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags, maxlen = get_pad_train_test_val(data_group, data)

# ------------------------------
# Step 4: Load GloVe Embeddings
# ------------------------------
def load_glove_embeddings(glove_file_path, embedding_dim):
    embeddings_index = {}
    with open(glove_file_path, encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print(f'Loaded {len(embeddings_index)} word vectors from GloVe.')
    return embeddings_index

def create_embedding_matrix(token2idx, embeddings_index, embedding_dim):
    vocab_size = len(token2idx) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in token2idx.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Load and prepare GloVe
glove_path = '/content/glove.6B.100d.txt'  # Ensure this path is correct
embedding_dim = 100
glove_embeddings = load_glove_embeddings(glove_path, embedding_dim)
embedding_matrix = create_embedding_matrix(token2idx, glove_embeddings, embedding_dim)

# ------------------------------
# Step 5: Define BiLSTM-LSTM Model with GloVe
# ------------------------------
def get_bilstm_lstm_model_with_glove(embedding_matrix, input_length, n_tags, embedding_dim):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_dim,
                        input_length=input_length,
                        weights=[embedding_matrix],
                        trainable=False))

    model.add(Bidirectional(LSTM(units=embedding_dim, return_sequences=True, dropout=0.2)))
    model.add(LSTM(units=embedding_dim, return_sequences=True, dropout=0.5))
    model.add(TimeDistributed(Dense(n_tags, activation="softmax")))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

n_tags = len(tag2idx)
model = get_bilstm_lstm_model_with_glove(embedding_matrix, maxlen, n_tags, embedding_dim)

# ------------------------------
# Step 6: Train the Model
# ------------------------------
def train_model(X, y, model):
    loss = []
    for i in range(25):
        hist = model.fit(X, y, batch_size=128, epochs=1, verbose=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

results = pd.DataFrame()
results['with_glove'] = train_model(train_tokens, np.array(train_tags), model)

# ------------------------------
# Step 7: Optional Visualization with spaCy (if desired)
# ------------------------------
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')


<ipython-input-15-5be7ed0af8d8>:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Sentence #'] = data['Sentence #'].fillna(method='ffill')


Train: 32372, Val: 10791, Test: 4796


FileNotFoundError: [Errno 2] No such file or directory: '/content/glove.6B.100d.txt'

In [ ]:
text = nlp('''On January 20, 2021, Joseph R. Biden Jr. was sworn in as the 46th President of the United States, taking over from Donald Trump. The inauguration ceremony took place in Washington, D.C., in front of the U.S. Capitol and was attended by dignitaries including Barack Obama, George W. Bush, and Bill Clinton.

During the event, Lady Gaga sang the national anthem, and Amanda Gorman delivered a powerful poem that went viral across social media platforms like Twitter and YouTube.

Meanwhile, in Silicon Valley, Sundar Pichai, the CEO of Google, announced a $1 billion investment in affordable housing in California. This followed Tim Cook's earlier commitment of $2.5 billion from Apple toward solving the housing crisis.

In Europe, Angela Merkel met with Emmanuel Macron at the Élysée Palace in Paris to discuss the European Union’s response to the ongoing energy crisis caused by reduced gas supplies from Russia. At the same time, NATO forces in Poland conducted joint military exercises with troops from Estonia and Lithuania.

Else
''')
displacy.render(text, style='ent', jupyter=True)

In [ ]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('/content/ner_model.h5')

# Now you can use model.predict(), model.evaluate(), or model.summary()
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 104, 100)            │       3,517,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 104, 200)            │         160,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 104, 100)            │         120,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 104, 17)             │           1,717 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,800,819 (14.50 MB)

 Trainable params: 282,917 (1.08 MB)

 Non-trainable params: 3,517,900 (13.42 MB)

 Optimizer params: 2 (12.00 B)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_tokens, np.array(train_tags),
    batch_size=128,
    epochs=25,
    validation_data=(val_tokens, np.array(val_tags)),
    verbose=1
)

import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


NameError: name 'train_tokens' is not defined

# **Asks Input**

WebApp

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install tensorflow
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 89.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from spacy import displacy

@st.cache(allow_output_mutation=True)
def load_keras_model():
    return load_model('/content/ner_model.h5')

@st.cache(allow_output_mutation=True)
def load_dicts():
    with open('/content/token2idx.pkl', 'rb') as f:
        token2idx = pickle.load(f)
    with open('/content/idx2tag.pkl', 'rb') as f:
        idx2tag = pickle.load(f)
    tag2idx = {tag: idx for idx, tag in idx2tag.items()}
    return token2idx, idx2tag, tag2idx

def preprocess(text, token2idx, maxlen):
    words = text.split()
    unk_token = token2idx.get('UNK', 0)
    seq = [token2idx.get(w, unk_token) for w in words]
    seq_padded = pad_sequences([seq], maxlen=maxlen, padding='post', value=token2idx.get('PAD', 0))
    return seq_padded, words

def decode_predictions(preds, idx2tag):
    pred_tags = []
    for pred in preds[0]:
        tag_idx = np.argmax(pred)
        pred_tags.append(idx2tag.get(tag_idx, 'O'))
    return pred_tags

def create_spacy_doc(text, tags):
    words = text.split()
    ents = []
    start = 0
    for word, tag in zip(words, tags):
        end = start + len(word)
        if tag != 'O':
            entity_type = tag.split('-')[-1]
            ents.append({"start": start, "end": end, "label": entity_type})
        start = end + 1
    doc_dict = {"text": text, "ents": ents, "title": None}
    return doc_dict

st.title("NER with BiLSTM Model & GloVe Embeddings")

model = load_keras_model()
token2idx, idx2tag, tag2idx = load_dicts()

maxlen = 75  # Use your model's max length

user_input = st.text_area("Enter text for NER", "Barack Obama was born in Hawaii.")

if st.button("Predict Entities"):
    seq_padded, words = preprocess(user_input, token2idx, maxlen)
    preds = model.predict(seq_padded)
    pred_tags = decode_predictions(preds, idx2tag)
    pred_tags = pred_tags[:len(words)]
    doc = create_spacy_doc(user_input, pred_tags)
    html = displacy.render(doc, style='ent', manual=True)
    st.markdown(html, unsafe_allow_html=True)


Overwriting app.py


In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 132.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.4 MB/s eta 0:00:00


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.124.168.111


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙⠹⠸⠼your url is: https://yummy-rats-spend.loca.lt
^C


In [ ]:
# # Install ngrok if not installed
# !pip install pyngrok
# from pyngrok import ngrok

# # Kill any previous streamlit instances
# !kill $(pgrep streamlit)

# # Start streamlit app in the background
# get_ipython().system_raw('streamlit run app.py &')

# # Open tunnel on port 8501 (default for streamlit)
# public_url = ngrok.connect(port='8501')
# print(f"Streamlit app URL: {public_url}")


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


ERROR:pyngrok.process.ngrok:t=2025-06-03T17:10:13+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-03T17:10:13+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-03T17:10:13+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.